In [1]:
import cv2
import Tkinter as tk
import tkFileDialog
import os
import numpy as np
import tensorflow as tf
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from Preprocessing import load_images
from keras.utils import np_utils
from sklearn import metrics as sk
from sklearn.metrics import cohen_kappa_score
np.random.seed(200)
%matplotlib inline

Using TensorFlow backend.


In [2]:
tk.Tk().withdraw()
path_Train=tkFileDialog.askdirectory()
print path_Train

/home/akash/Skin-Lesion-Analysis-Towards-Melanoma-Detection/Training Images


In [3]:
X_Train=load_images(path_Train)

In [4]:
X_Train=np.array(X_Train)
X_Train.shape

(2000, 64, 64, 3)

In [5]:
X_Train=np.reshape(X_Train,[-1,np.prod(X_Train.shape[1:])])
X_Train.shape

(2000, 12288)

In [6]:
tk.Tk().withdraw()
path_Test=tkFileDialog.askdirectory()
print path_Test

/home/akash/Skin-Lesion-Analysis-Towards-Melanoma-Detection/Testing Images


In [7]:
X_Test=load_images(path_Test)
X_Test=np.array(X_Test)
X_Test.shape

(600, 64, 64, 3)

In [8]:
X_Test=np.reshape(X_Test,[-1,np.prod(X_Test.shape[1:])])
X_Test.shape

(600, 12288)

In [9]:
data_Train=pd.read_csv("/home/akash/Skin-Lesion-Analysis-Towards-Melanoma-Detection/ISIC-2017_Training_Part3_GroundTruth.csv")
data_Test=pd.read_csv("/home/akash/Skin-Lesion-Analysis-Towards-Melanoma-Detection/ISIC-2017_Test_v2_Part3_GroundTruth.csv")

In [10]:
data_Train=data_Train.iloc[0:2000,1]
data_Test=data_Test.iloc[0:600,1]
print data_Train.shape
print data_Test.shape

(2000,)
(600,)


In [11]:
y_Train=data_Train
y_Test=data_Test

In [12]:
y_Train=np.array(y_Train)
y_Test=np.array(y_Test)

In [13]:
y_Train = np_utils.to_categorical(data_Train)
y_Test = np_utils.to_categorical(data_Test)
print y_Train.shape
print y_Test.shape

(2000, 2)
(600, 2)


In [14]:
def center_normalize(x):
    return (x-np.mean(x))/np.std(x)
X_Train=center_normalize(X_Train)
X_Test=center_normalize(X_Test)

In [15]:
X_Test.shape

(600, 12288)

In [16]:
learning_rate=0.01
training_iterations=50000
batch_size=100
display_step=10
logs_path = '/tmp/tensorflow_logs/example'
n_input=64*64*3 # Data input 
n_classes=2 # Classes
dropout=0.75

x=tf.placeholder("float",[None,n_input])
y=tf.placeholder("float",[None,n_classes])
Drop_prob=tf.placeholder(tf.float32)

In [17]:
# Create some wrappers for simplicity
def conv2d(x, W, b, strides=1):
    # Conv2D wrapper, with bias and relu activation
    x = tf.nn.conv2d(x, W, strides=[1, strides, strides, 1], padding='SAME')
    x = tf.nn.bias_add(x, b)
    return tf.nn.relu(x)


def maxpool2d(x, k=2):
    # MaxPool2D wrapper
    return tf.nn.max_pool(x, ksize=[1, k, k, 1], strides=[1, k, k, 1],
                          padding='SAME')


# Create model
def conv_net(x, weights, biases, dropout):
    # Reshape input picture
    x = tf.reshape(x, shape=[-1, 64, 64, 3])

    # Convolution Layer
    conv1 = conv2d(x, weights['wc1'], biases['bc1'])
    # Max Pooling (down-sampling)
    conv1 = maxpool2d(conv1, k=2)

    # Convolution Layer
    conv2 = conv2d(conv1, weights['wc2'], biases['bc2'])
    # Max Pooling (down-sampling)
    conv2 = maxpool2d(conv2, k=2)

    # Fully connected layer
    # Reshape conv2 output to fit fully connected layer input
    fc = tf.reshape(conv2, [-1, weights['wd1'].get_shape().as_list()[0]])
    fc1 = tf.add(tf.matmul(fc, weights['wd1']), biases['bd1'])
    fc1 = tf.nn.relu(fc1)
    # Apply Dropout
    fc1 = tf.nn.dropout(fc1, dropout)

    # Output, class prediction
    out = tf.add(tf.matmul(fc1, weights['out']), biases['out'])
    return out

In [18]:
# Store layers weight & bias
weights = {
    # 5x5 conv, 1 input, 32 outputs
    'wc1': tf.Variable(tf.random_normal([3, 3, 3, 16])),
    # 5x5 conv, 32 inputs, 64 outputs
    'wc2': tf.Variable(tf.random_normal([3, 3, 16, 16])),
    # fully connected, 16*16*64 inputs, 4096 outputs
    'wd1': tf.Variable(tf.random_normal([16*16*16, 12288])),
    # 4096 inputs, 3 outputs (class prediction)
    'out': tf.Variable(tf.random_normal([12288, n_classes]))
}

biases = {
    'bc1': tf.Variable(tf.random_normal([16])),
    'bc2': tf.Variable(tf.random_normal([16])),
    'bd1': tf.Variable(tf.random_normal([12288])),
    'out': tf.Variable(tf.random_normal([n_classes]))
}

# Construct model
with tf.name_scope("Model"):
    pred= conv_net(x, weights, biases, Drop_prob)

# Define loss and optimizer
with tf.name_scope("loss"):
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=y))
with tf.name_scope("Adam"):
    optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

# Evaluate model
with tf.name_scope("Accuracy"):
    correct_pred = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# Initializing the variables
init = tf.global_variables_initializer()
# summaries
tf.summary.scalar("loss",cost)
tf.summary.scalar("accuracy",accuracy)
merged_summary_op=tf.summary.merge_all()

In [19]:
# launch the graph
with tf.Session() as sess:
    sess.run(init)
    step=1
    # Training 
    summary_writer=tf.summary.FileWriter(logdir=logs_path,graph=tf.get_default_graph())
    while step*batch_size<training_iterations:
        train_idx=np.random.randint(X_Train.shape[0],size=batch_size)
        batch_x_Train=X_Train[train_idx,:]
        batch_y_Train=y_Train[train_idx]
        _,c,summary=sess.run([optimizer,cost,merged_summary_op],feed_dict={x:batch_x_Train,y:batch_y_Train,Drop_prob:dropout})
        summary_writer.add_summary(summary,step*batch_size)
        if step%display_step==0:
            loss,acc=sess.run([cost,accuracy],feed_dict={x:batch_x_Train,y:batch_y_Train,Drop_prob:1.})
        
            print "iters:"+ str(step*batch_size)+",Minibatch Loss=" + \
                "{:.6f}".format(loss)+",Training Accuracy="+\
                "{:.5f}".format(acc)
        step+=1
    print "Optimization Finished"
    test_idx=np.random.randint(X_Test.shape[0],size=batch_size)
    batch_x_Test=X_Test[test_idx,:]
    batch_y_Test=y_Test[test_idx]
    print "Testing accuracy",\
        sess.run(accuracy,feed_dict={x:batch_x_Test,y:batch_y_Test,Drop_prob:1.})

iters:1000,Minibatch Loss=92402.851562,Training Accuracy=0.66000
iters:2000,Minibatch Loss=15235.757812,Training Accuracy=0.81000
iters:3000,Minibatch Loss=13821.407227,Training Accuracy=0.85000
iters:4000,Minibatch Loss=5860.616211,Training Accuracy=0.86000
iters:5000,Minibatch Loss=4305.635254,Training Accuracy=0.81000
iters:6000,Minibatch Loss=1346.981689,Training Accuracy=0.89000
iters:7000,Minibatch Loss=2519.657715,Training Accuracy=0.85000
iters:8000,Minibatch Loss=1113.013062,Training Accuracy=0.92000
iters:9000,Minibatch Loss=1483.367676,Training Accuracy=0.91000
iters:10000,Minibatch Loss=536.172119,Training Accuracy=0.93000
iters:11000,Minibatch Loss=458.709290,Training Accuracy=0.94000
iters:12000,Minibatch Loss=382.804779,Training Accuracy=0.96000
iters:13000,Minibatch Loss=1055.420410,Training Accuracy=0.94000
iters:14000,Minibatch Loss=81.320709,Training Accuracy=0.98000
iters:15000,Minibatch Loss=63.781754,Training Accuracy=0.99000
iters:16000,Minibatch Loss=73.752792,T